In [1]:
import numpy as np
import h5py
import sys
import os

sys.path.append("../2_train_models")
from file_configs import MergedFilesConfig

from common_functions import load_coords

In [2]:
# specify what set of models to look at
cell_type = "K562"

# these usually don't change
model_type = "strand_merged_umap"
data_type = "procap"

# size of the model inputs and outputs
in_window = 2114
slice_len = 1000

In [16]:
# download the hg38 repeatmasker/repbase annotation of all repeats from UCSC

! wget https://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/rmsk.txt.gz -O "/users/kcochran/projects/procapnet/annotations/rmsk.txt.gz"
! gunzip "/users/kcochran/projects/procapnet/annotations/rmsk.txt.gz"
! awk -v OFS="\t" '{ print $6, $7, $8, $11, $12, $13 }' "/users/kcochran/projects/procapnet/annotations/rmsk.txt" > "/users/kcochran/projects/procapnet/annotations/rmsk.bed"
! rm "/users/kcochran/projects/procapnet/annotations/rmsk.txt"

--2024-03-12 21:23:20--  https://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/rmsk.txt.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155633856 (148M) [application/x-gzip]
Saving to: ‘/users/kcochran/projects/procapnet/annotations/rmsk.txt.gz’

/users/kcochran/pro 100%[===================>] 148.42M   106MB/s    in 1.4s    

2024-03-12 21:23:22 (106 MB/s) - ‘/users/kcochran/projects/procapnet/annotations/rmsk.txt.gz’ saved [155633856/155633856]



In [17]:
repeat_masker_bed = "/users/kcochran/projects/procapnet/annotations/rmsk.bed"

In [18]:
! head $repeat_masker_bed

chr1	10000	10468	(TAACCC)n	Simple_repeat	Simple_repeat
chr1	10468	11447	TAR1	Satellite	telo
chr1	11504	11675	L1MC5a	LINE	L1
chr1	11677	11780	MER5B	DNA	hAT-Charlie
chr1	15264	15355	MIR3	SINE	MIR
chr1	15797	15849	(TGCTCC)n	Simple_repeat	Simple_repeat
chr1	16712	16744	(TGG)n	Simple_repeat	Simple_repeat
chr1	18906	19048	L2a	LINE	L2
chr1	19971	20405	L3	LINE	CR1
chr1	20530	20679	Plat_L3	LINE	CR1


In [3]:
# Load the config object (filepath holder) for these models
config = MergedFilesConfig(cell_type, model_type, data_type)

proj_dir = config.proj_dir
tmp_data_dir = "te_enrichment_files/"
os.makedirs(tmp_data_dir, exist_ok=True)

# load the chrom, start, end info for all peaks
coords = load_coords(config.all_peak_path, in_window)

In [6]:
# for each pattern we want to look at, we need to:
#  1) load all the genomic sequences supporting this pattern (seqlets)
#  2) write their coordinates to a bed file
#  3) get enrichment stats for those coordinates vs. repeat annotations

def extract_genomic_coords_at_seqlets(modisco_results_path, pattern_i,
                                      coords, in_window, slice_len):
    
    # this function figures out the genomic coordinates for each sequence
    # that modisco clustered into a pattern of interest
        
    def seqlet_coord_to_input_coord(seqlet_coord):
        # seqlet coordinates are by default w.r.t. the model output window;
        # this adjusts coords to be w.r.t. the whole peak / model input window
        return seqlet_coord + ((in_window - slice_len) // 2)
    
    def load_seqlets(modisco_results, pattern_i):
        patterns_grp = modisco_results["pos_patterns"]
        pattern_grp = patterns_grp["pattern_" + str(pattern_i)]
        return pattern_grp["seqlets"]
        
    # open the modisco hdf5 results file object
    modisco_results = h5py.File(modisco_results_path, "r")
    
    # hunt through the hdf5 file structure for the seqlet info
    seqlets = load_seqlets(modisco_results, pattern_i)
    
    coord_indexes = seqlets["example_idx"][:]
    seqlet_starts = seqlets["start"][:]
    seqlet_ends = seqlets["end"][:]
    seqlet_rcs = seqlets["is_revcomp"][:]
    
    # adjust seqlet coordinates so that they're normal genomic coords
    
    input_starts = seqlet_coord_to_input_coord(seqlet_starts)
    input_ends = seqlet_coord_to_input_coord(seqlet_ends)

    genomic_coords = []
    for coord_index, input_start, input_end in zip(coord_indexes, input_starts, input_ends):
        chrom, peak_start = coords[coord_index][:2]
        genomic_coords.append((chrom, peak_start + input_start, peak_start + input_end))

    modisco_results.close()
        
    return genomic_coords


def write_coords_to_bed(bed_filepath, coords):
    to_write = "\n".join(["\t".join([str(i) for i in coord]) for coord in coords])
    
    if bed_filepath.endswith(".gz"):
        with gzip.open(bed_filepath, "w") as f:
            f.write(to_write.encode())
    else:
        with open(bed_filepath, "w") as f:
            f.write(to_write)

            
def write_all_seqlet_coords_to_beds(task, tmp_data_dir = tmp_data_dir):
    # this is specific to K562
    
    if task == "profile":
        # labels for each of the patterns stored in the modisco results file
        pattern_labels = ["BRE/SP", "CA-Inr", "ETS", "NFY", "NRF1",
                          "ATF1", "TATA", "THAP11", "YY1", "AP1",
                          "DPR_CGG", "DPR_CGG", "DPR_CGG", "TA-Inr", "DPR_CGG",
                          "CTCF", "DPR_CGG", "NRF1-like", "DPR_CGG", "ZBTB33",
                          "DPR_CGG", "TCT", "BRE/SP_TE", "TATATA", "CA-Inr_TE",
                          "DPR_CGG", "DPR_CGG", "TATA_TE", "CA-Inr_dimer", "DPR_CGG", "DPR_CGG",
                          "ZBTBT33_TATA_TE", "TE", "GC-rich", "CA-Inr_TE", "YY1-like",
                          "NFY_TE", "ZBTB33_TE", "ETS_TE", "ETS_dimer", "NFY_C-rich",
                          "ETS_CA-Inr_dimer", "YY1-like"]

        # modisco results hdf5 object containing the motifs/patterns and subpatterns
        modisco_results_path = config.modisco_profile_results_path
    else:
        pattern_labels = ["ETS", "BRE/SP", "NRF1", "ETS-like", "NFY", "ATF1",
                          "CpG", "AP1", "CpG_spacing", "CpG_spacing", "THAP11",
                          "CpG_spacing", "CpG_spacing", "CpG_spacing",
                          "ZBTB33", "CpG_spacing", "BRE/SP_TE", "CpG_spacing", "CpG_spacing",
                          "THAP11-like", "CpG_spacing", "BRE/SP_TE", "ZBTB33_TE", "CpG_spacing",
                          "BRE/SP_TE", "TE", "BRE/SP_TE", "TE", "TE",
                          "ETS-like", "BRE/SP_TE", "Unknown", "NFY_TE"]

        modisco_results_path = config.modisco_counts_results_path
    
    
    for pattern_i, pattern_label in enumerate(pattern_labels):
        if "TE" in pattern_label:

            seqlet_coords = extract_genomic_coords_at_seqlets(modisco_results_path, pattern_i,
                                                              coords, in_window, slice_len)

            bed_filepath = tmp_data_dir + task + ".pattern_" + str(pattern_i) + ".seqlets.bed"
            write_coords_to_bed(bed_filepath, seqlet_coords)
            
            
for task in ["profile", "counts"]:
    write_all_seqlet_coords_to_beds(task)

In [9]:
# then, run repeat enrichment analysis on each of the bed files
# (for each of the patterns that might be a TE, for both model tasks)

bed_paths = [tmp_data_dir + fpath for fpath in os.listdir(tmp_data_dir)]

In [75]:
import subprocess

def run_bedtools_intersect(filepath_a, filepath_b, dest_filepath, other_args=[]):
    cmd = ["bedtools", "intersect"]
    cmd += ["-a", filepath_a]
    cmd += ["-b", filepath_b]
    for arg in other_args:
        cmd += [arg]
        
    with open(dest_filepath, "w") as outf:
        subprocess.call(cmd, stdout=outf)
        

def calc_repeat_overlap_frac(results_fpath, seqlets_bed):
    # returns the fractions of seqlets that overlapped any repeat annotation
    with open(seqlets_bed) as f:
        num_seqlets = sum([1 for line in f])
    
    with open(results_fpath) as resultsf:
        hits = [line.strip().split() for line in resultsf]
        
    # don't double-count stuff that overlapped two annotations?
    hit_coords = set([tuple(hit[:3]) for hit in hits])
    
    return len(hit_coords) / num_seqlets
    
    
def get_most_common_repeat_type(results_fpath, repeat_col):
    # the repeat masker file has 3 columns of repeat labels:
    # -3 is the most specific label / subfamily,
    # -2 is the least specific or the overall class (LINE, SINE, LTR...)
    # -1 is the middle or family (e.g. Alu)
    
    assert repeat_col in [-3, -2, -1], repeat_col
    
    with open(results_fpath) as resultsf:
        hits = [line.strip().split() for line in resultsf]
    
    # for every repeat type overlapped, count how often it was overlapped
    repeat_types, repeat_type_counts = np.unique([hit[repeat_col] for hit in hits],
                                                 return_counts=True)
    
    # sort repeat types overlapped so the most common is first
    repeat_types_and_counts = sorted(list(zip(repeat_type_counts, repeat_types)),
                                     reverse=True)
    
    # calc the fraction of repeat overlaps that came from this most-common one
    total = sum([count[0] for count in repeat_types_and_counts])
    frac_most_common_repeat_type = repeat_types_and_counts[0][0] / total
    
    # return name of most common repeat type, and fraction of overlaps it was
    return repeat_types_and_counts[0][1], frac_most_common_repeat_type
    

def run_bedtools_intersect_calc_repeat_overlap(rmsk_bed, seqlets_bed):
    assert not seqlets_bed.endswith(".gz") # probably doesn't work with gzipped beds
    
    tmpf = "tmp.bed"
    
    run_bedtools_intersect(seqlets_bed, rmsk_bed, tmpf, other_args=["-wa", "-wb"])
        
    repeat_overlap_frac = calc_repeat_overlap_frac(tmpf, seqlets_bed)
    
    repeat_class_hit = get_most_common_repeat_type(tmpf, -2)
    repeat_family_hit = get_most_common_repeat_type(tmpf, -1)
        
    return repeat_overlap_frac, repeat_class_hit, repeat_family_hit

    
def sort_bed_paths(bed_paths):
    # orders filenames by the pattern number that is inside the filename;
    # would make sense to plot in an order consistent with the supp fig of patterns
    
    sort_by = []
    for bed_path in bed_paths:
        pattern_num = int(bed_path.split("pattern_")[-1].split(".")[0])
        sort_by.append(pattern_num)
        
    bed_paths = np.array(bed_paths)[np.argsort(sort_by)[::-1]]
    return bed_paths
    

def make_table(repeat_masker_bed, bed_paths):
    # this function prints a full LateX table you can copy-paste into overleaf
    
    # table preamble stuff
    print(r'\hline')
    print(r'TE Pattern & Repeat Overlap (\%) & Most Common Repeat Family & Fraction of Repeat Overlap Matching Family (\%) \\')
    print(r'\hline')
    print(r'\endhead')
    print(r'\hline')
    print(r'\endfoot')
    print(r'\hline')
    print(r'\endlastfoot')

    # separate the patterns by task, sort in numeric order
    prof_bed_paths = sort_bed_paths([bed_path for bed_path in bed_paths if "profile" in bed_path])
    counts_bed_paths = sort_bed_paths([bed_path for bed_path in bed_paths if "counts" in bed_path])

    # first, list out repeat enrichment of profile patterns
    # (formatted to be like a row in a table)
    for pattern_i, bed_path in enumerate(prof_bed_paths):
        pattern_label = "Profile TE Pattern " + str(pattern_i + 1)
        row_str = pattern_label + " & "
        
        repeat_overlap_results = run_bedtools_intersect_calc_repeat_overlap(repeat_masker_bed, bed_path)
        repeat_overlap_frac, repeat_class_hit, repeat_family_hit = repeat_overlap_results
        
        row_str += "%0.f" % (repeat_overlap_frac * 100) + r'\% & '
        
        row_str += repeat_family_hit[0] + " (" + repeat_class_hit[0] + ") & "
        row_str += "%0.f" % (repeat_family_hit[1] * 100) + r'\% \\'
        
        print(row_str)
        
    print(r'\hline')
        
    # then do the same for counts patterns
    for pattern_i, bed_path in enumerate(counts_bed_paths):
        pattern_label = "Counts TE Pattern " + str(pattern_i + 1)
        row_str = pattern_label + " & "
        
        repeat_overlap_results = run_bedtools_intersect_calc_repeat_overlap(repeat_masker_bed, bed_path)
        repeat_overlap_frac, repeat_class_hit, repeat_family_hit = repeat_overlap_results
        
        row_str += "%0.f" % (repeat_overlap_frac * 100) + r'\% & '
        
        row_str += repeat_family_hit[0] + " (" + repeat_class_hit[0] + ") & "
        
        ########################
        if pattern_i != len(counts_bed_paths) - 1:
            row_str += "%0.f" % (repeat_family_hit[1] * 100) + r'\% \\'
        else:
            row_str += "%0.f" % (repeat_family_hit[1] * 100) + r'\%\label{Tab2}\\'
            
        print(row_str)
    
    
make_table(repeat_masker_bed, bed_paths)

\hline
TE Pattern & Repeat Overlap (\%) & Most Common Repeat Family & Fraction of Repeat Overlap Matching Family (\%) \\
\hline
\endhead
\hline
\endfoot
\hline
\endlastfoot
Profile TE Pattern 1 & 82\% & ERV1 (LTR) & 100\% \\
Profile TE Pattern 2 & 84\% & ERVL-MaLR (LTR) & 97\% \\
Profile TE Pattern 3 & 81\% & ERV1 (LTR) & 100\% \\
Profile TE Pattern 4 & 91\% & ERV1 (LTR) & 99\% \\
Profile TE Pattern 5 & 75\% & Alu (SINE) & 93\% \\
Profile TE Pattern 6 & 81\% & ERVL-MaLR (LTR) & 100\% \\
Profile TE Pattern 7 & 70\% & ERV1 (LTR) & 100\% \\
Profile TE Pattern 8 & 78\% & Alu (SINE) & 94\% \\
Profile TE Pattern 9 & 67\% & ERV1 (LTR) & 93\% \\
Counts TE Pattern 1 & 96\% & ERV1 (LTR) & 100\% \\
Counts TE Pattern 2 & 82\% & L1 (LINE) & 97\% \\
Counts TE Pattern 3 & 82\% & ERVL-MaLR (LTR) & 98\% \\
Counts TE Pattern 4 & 73\% & L1 (LINE) & 96\% \\
Counts TE Pattern 5 & 95\% & ERVL-MaLR (LTR) & 96\% \\
Counts TE Pattern 6 & 91\% & Alu (SINE) & 97\% \\
Counts TE Pattern 7 & 85\% & ERV1 (LTR) & 86\